In [2]:
%%time

from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
from time import sleep
from threading import Thread

import pandas as pd

from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

session = Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
CPU times: user 1.3 s, sys: 259 ms, total: 1.56 s
Wall time: 1.74 s


In [3]:
test_data = pd.read_csv("test_data/test_data.csv", header=None)
test_data

,0,1,2,3,4
0,0.241110,5,0.0,9.24,-1.0
1,0.191735,5,1.0,9.31,-1.0
2,0.155147,5,2.0,9.72,-1.0
3,0.135726,5,3.0,9.18,-1.0
4,0.126615,5,4.0,6.73,-1.0
...,...,...,...,...,...
644,0.347004,3,20.0,5.94,-1.0
645,0.334706,3,21.0,5.03,-1.0
646,0.299760,3,22.0,4.10,-1.0
647,0.252274,3,23.0,3.93,-1.0


In [8]:
import json

def dump_to_jsonlines_file(df, filename):
    with open(filename, "w") as f:
        for _, row in df.iterrows():
            # Assuming the first column is the label and the rest are features
            sample = {"features": row[1:].tolist(), "label": row[0]}
            print(json.dumps(sample), file=f)

# Replace 'test_data' with your actual DataFrame
dump_to_jsonlines_file(test_data, "test_data/test_data.jsonl")

In [10]:
!head -n 5 test_data/test_data.jsonl

{"features": [5.0, 0.0, 9.24, -1.0], "label": 0.2411102}
{"features": [5.0, 1.0, 9.31, -1.0], "label": 0.19173479}
{"features": [5.0, 2.0, 9.72, -1.0], "label": 0.15514688}
{"features": [5.0, 3.0, 9.18, -1.0], "label": 0.13572587}
{"features": [5.0, 4.0, 6.73, -1.0], "label": 0.12661467}


In [11]:
validation_data = pd.read_csv("test_data/validation_data.csv", header=None)
validation_data

,0,1,2,3,4
0,0.259011,2,0.0,7.76,5.0
1,0.227850,2,1.0,6.90,5.0
2,0.190840,2,2.0,6.59,5.0
3,0.158080,2,3.0,6.76,5.0
4,0.140162,2,4.0,6.74,5.0
...,...,...,...,...,...
739,0.392644,4,19.0,7.08,-1.0
740,0.370253,4,20.0,7.53,-1.0
741,0.345889,4,21.0,7.68,-1.0
742,0.312565,4,22.0,8.31,-1.0


In [12]:
# Replace 'validation_data' with your actual DataFrame
dump_to_jsonlines_file(validation_data, "test_data/validation_data.jsonl")

In [13]:
!head -n 5 test_data/validation_data.jsonl

{"features": [2.0, 0.0, 7.76, 5.0], "label": 0.259011}
{"features": [2.0, 1.0, 6.9, 5.0], "label": 0.22785021}
{"features": [2.0, 2.0, 6.59, 5.0], "label": 0.1908404}
{"features": [2.0, 3.0, 6.76, 5.0], "label": 0.1580797}
{"features": [2.0, 4.0, 6.74, 5.0], "label": 0.14016196}
